In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark=SparkSession.builder.appName("Dunhumbby Practise").getOrCreate()
rawDF=spark.sql("select * from aadhaar_data_csv")

from pyspark.sql.functions import concat_ws, date_format, lit, unix_timestamp, to_date
from datetime import datetime
from pyspark.sql import functions as f

columnList=["date","registrar","private_agency","state","district","sub_district","pin","gender","age","aadhaar_generated","rejected","mobile_number","email_id"]
inputDF=rawDF.rdd.toDF(columnList)
inputDF=inputDF.withColumn("date", to_date(unix_timestamp(inputDF["date"],"yyyyMMdd").cast("timestamp")))
inputDF.persist()


+--------------------+------+
 registrar| count|
+--------------------+------+
Govt of Andhra Pr...| 47489|
 UT Of Daman and Diu| 1786|
Govt of Madhya Pr...| 3582|
Punjab National Bank| 3303|
NSDL e-Governance...|122709|
 IDBI Bank ltd| 9218|
 Delhi - North DC| 4372|
 Govt of Maharashtra|165780|
 Indiapost| 22116|
 FCS Govt of Punjab| 29937|
Govt of Sikkim - ...| 783|
 Delhi - NE DC| 1516|
 FCR Govt of Haryana| 7959|
 IGNOU| 124|
Principal Revenue...| 2157|
 Syndicate Bank| 25|
Indian Overseas Bank| 52|
 Delhi - East DC| 1957|
Department of Inf...| 3772|
 Bank of Maharashtra| 2723|
+--------------------+------+
only showing top 20 rows

+--------------------+---------------+-------------------+
 state|no_of_districts|no_of_sub_districts|
+--------------------+---------------+-------------------+
 Nagaland| 9| 51|
 Karnataka| 44| 217|
 Odisha| 33| 357|
 Kerala| 15| 100|
 Tamil Nadu| 40| 238|
 Chhattisgarh| 34| 148|
 Andhra Pradesh| 29| 1110|
 Lakshadweep| 1| 5|
 Madhya Pradesh| 50| 273|
 Punjab| 25| 99|
 Manipur| 9| 43|
 Goa| 7| 20|
 Mizoram| 7| 13|
Dadra and Nagar H...| 1| 2|
 Himachal Pradesh| 13| 206|
 Puducherry| 4| 15|
 Haryana| 21| 74|
 Jammu and Kashmir| 24| 81|
 Jharkhand| 29| 234|
 Arunachal Pradesh| 16| 39|
+--------------------+---------------+-------------------+
only showing top 20 rows

In [2]:
#show number of registrars
inputDF.groupBy(inputDF["registrar"]).count().show(3)

#Show Number of district, sub-district in each state
inputDF.groupBy(inputDF["state"]).agg(f.countDistinct("district").alias("no_of_districts"), f.countDistinct("sub_district").alias("no_of_sub_districts")).show()

+--------------------+------+
 registrar| count|
+--------------------+------+
Govt of Andhra Pr...| 47489|
 UT Of Daman and Diu| 1786|
Govt of Madhya Pr...| 3582|
Punjab National Bank| 3303|
NSDL e-Governance...|122709|
 IDBI Bank ltd| 9218|
 Delhi - North DC| 4372|
 Govt of Maharashtra|165780|
 Indiapost| 22116|
 FCS Govt of Punjab| 29937|
Govt of Sikkim - ...| 783|
 Delhi - NE DC| 1516|
 FCR Govt of Haryana| 7959|
 IGNOU| 124|
Principal Revenue...| 2157|
 Syndicate Bank| 25|
Indian Overseas Bank| 52|
 Delhi - East DC| 1957|
Department of Inf...| 3772|
 Bank of Maharashtra| 2723|
+--------------------+------+
only showing top 20 rows

+--------------------+---------------+-------------------+
 state|no_of_districts|no_of_sub_districts|
+--------------------+---------------+-------------------+
 Nagaland| 9| 51|
 Karnataka| 44| 217|
 Odisha| 33| 357|
 Kerala| 15| 100|
 Tamil Nadu| 40| 238|
 Chhattisgarh| 34| 148|
 Andhra Pradesh| 29| 1110|
 Lakshadweep| 1| 5|
 Madhya Pradesh| 50| 273|
 Punjab| 25| 99|
 Manipur| 9| 43|
 Goa| 7| 20|
 Mizoram| 7| 13|
Dadra and Nagar H...| 1| 2|
 Himachal Pradesh| 13| 206|
 Puducherry| 4| 15|
 Haryana| 21| 74|
 Jammu and Kashmir| 24| 81|
 Jharkhand| 29| 234|
 Arunachal Pradesh| 16| 39|
+--------------------+---------------+-------------------+
only showing top 20 rows

In [3]:
#show total number of male population in each state
maleDF=inputDF.filter(inputDF["gender"]=="M")
maleDF.groupBy(inputDF["state"]).agg(f.count("gender").alias("Male_Count")).show()

+--------------------+----------+
 state|Male_Count|
+--------------------+----------+
 Nagaland| 792|
 Karnataka| 44916|
 Odisha| 12213|
 Kerala| 41801|
 Tamil Nadu| 28359|
 Chhattisgarh| 10189|
 Andhra Pradesh| 74748|
 Lakshadweep| 9|
 Madhya Pradesh| 56623|
 Punjab| 25494|
 Manipur| 3651|
 Goa| 2780|
 Mizoram| 86|
Dadra and Nagar H...| 91|
 Himachal Pradesh| 10032|
 Puducherry| 532|
 Jammu and Kashmir| 2715|
 Haryana| 21890|
 Jharkhand| 32005|
 Arunachal Pradesh| 273|
+--------------------+----------+
only showing top 20 rows

In [4]:
#Count total number of private agency in each state
privateDF=inputDF.filter("private_agency is not null")
agencyByState=inputDF.groupBy(inputDF["state"],inputDF["private_agency"]).count()
agencyByState.show(3)


from pyspark.sql.functions import col
#Show most active private agency in each state
df=inputDF.groupBy(inputDF["state"],inputDF["private_agency"]).count()
maxAgencyDF=df.groupBy(col("state")).agg(f.max(col("count")).alias("count"))
mostActiveAgency=maxAgencyDF.alias('a').join(agencyByState.alias('b'), (col("a.state")==col("b.state")) & (col("b.count")==col("a.count"))).select("a.state","private_agency","a.count")
mostActiveAgency.show()
#count unique private agency in each state

+-----------+--------------------+-----+
 state| private_agency|count|
+-----------+--------------------+-----+
 Haryana|Conatus Infocom P...| 3|
West Bengal|FINANCIAL INFORMA...| 45|
 Haryana|Vakrangee Softwar...| 4184|
+-----------+--------------------+-----+
only showing top 3 rows

+-----------------+--------------------+-----+
 state| private_agency|count|
+-----------------+--------------------+-----+
 Karnataka| Origin ITFS Pvt Ltd|10135|
 Himachal Pradesh| Wipro Ltd|10730|
Jammu and Kashmir|COMTECHINFO SOLUT...| 1964|
 Haryana|Karvy Data Manage...| 6959|
 Tripura| BASIX| 1313|
 Manipur|Manipur Electroni...| 3304|
 West Bengal|Emdee Digitronics...|13832|
 Jharkhand| Wipro Ltd|14323|
 Kerala| Akshaya|40573|
 Goa|MARS Telecom Syst...| 4492|
 Tamil Nadu|Swathy Smartcards...|24919|
 Chhattisgarh| CHIPS| 3848|
 Madhya Pradesh|Vakrangee Softwar...|20985|
 Rajasthan|Vakrangee Softwar...|27547|
Arunachal Pradesh|Webx Technologies...| 248|
 Others|India Computer Te...| 231|
 Andhra Pradesh| 4G INFORMATICS|15685|
 Mizoram|Integrated System...| 134|
 Daman and Diu|Alankit Assignmen...| 958|
 Telangana| Directorate of ESD| 2737|
+-----------------+--------------------+-----+
only showing top 20 rows

In [5]:
totalAadharDF=inputDF.groupBy(inputDF["state"]).agg(f.sum("aadhaar_generated").alias("total_number"))
totalAadharDF.orderBy(totalAadharDF["total_number"].desc()).show(3, False)

+--------------+------------+
state |total_number|
+--------------+------------+
Maharashtra |951201.0 |
Uttar Pradesh |385463.0 |
Andhra Pradesh|270055.0 |
+--------------+------------+
only showing top 3 rows

In [6]:
inputDF.select("pin").distinct().count()

inputDF.filter((inputDF["state"].contains("Uttar Pradesh")) | (inputDF["state"].contains("Maharashtra"))).groupBy(inputDF["state"]).agg(f.count("rejected")).show()

+-------------+---------------+
 state|count(rejected)|
+-------------+---------------+
 Maharashtra| 285326|
Uttar Pradesh| 209410|
+-------------+---------------+

In [7]:
maleDF=inputDF.filter(inputDF["gender"]=="M").groupBy(inputDF["state"]).agg(f.count("aadhaar_generated").alias("male aadhar"))
totalDF=inputDF.groupBy(inputDF["state"]).agg(f.count("aadhaar_generated").alias("total aadhar"))
joinDF=totalDF.join(maleDF, totalDF["state"]==maleDF["state"]).select(maleDF["state"],"total aadhar","male aadhar")
joinWithPercent=joinDF.withColumn("percent Data", lit((joinDF["male aadhar"]/joinDF["total aadhar"])*100))
top3DF=joinWithPercent.orderBy(joinWithPercent["percent Data"].desc()).limit(3)
top3DF.persist()

Out[ 6 ]: DataFrame[state: string, total aadhar: bigint, male aadhar: bigint, percent Data: double]

In [8]:
stateList=top3DF.select("state").rdd.map(list).collect()
#femaleDF=inputDF.filter(inputDF["state"].isin(stateList) & inputDF["gender"]=="F")
#femaleDF.show()
